In [ ]:
import numpy as np
import pandas as pd
import ast
import re
from openai import OpenAI, BadRequestError
from kiwipiepy.utils import Stopwords # 한국어 자연어 처리 패키지
from kiwipiepy import Kiwi
import faiss
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAI as OpenAI2

from hard_prompt import generate_welcome_message, generate_state_message, classify_prompt, purchase_prompt, get_chat_prompt, RAG_prompt, matching_rag_prompt
api_key = api_key

In [226]:
class base_f:
    def __init__(self, api_key):
        self.api_key = api_key
        self.client = OpenAI(api_key=api_key)
        self.df = pd.read_pickle('20240222_JJAgentData.pkl')
        self.unmatched_msg = "죄송합니다, 현재 요청하신 매칭 조건에 적합한 사용자를 찾지 못하였습니다. 원하시는 다른 매칭이 있으신가요?"
    
    # 새로운 엑셀 파일 데이터 추가
    def add_new_data(self, excel_path):
        new_data = pd.read_excel(excel_path)
        new_data['Embeddings'] = new_data['메시지'].apply(lambda x: self.get_embedding(x))
        pd.concat([self.df, new_data], ignore_index=True).to_pickle('2024nnnn_JJAgentData.pkl')

    # 매칭이 필요한 대화내용 저장 (데이터프레임 형태로)
    def update(self, msg):
        input_vector = self.get_embedding(msg)
        self.df.loc[len(self.df)] = (self.nickname, msg, input_vector)
        self.df.to_pickle('./data_for_update.pkl')

    # 연결/결제 구분        
    def classify_process(self, text):
        prompt = f"""If the sentence({text}) seems to buy or sell something, only print 'Buy&Sell'.
                    If the sentence seems to be connected or matched someone, only print 'Match'.
                    If you cannot judge, only print out 'Match'."""
        response = self.client.chat.completions.create(
                        model = "gpt-4-1106-preview",
                        messages=[{"role": "system", "content": prompt}],
                        temperature = 0
                    )
        response = response.choices[0].message.content
        return response
    
    # 대화내용으로부터 상품명, 가격, 수량 추출
    def purchase_keyword(self, api_key, msg):
        purchase_prompt
        example_prompt = PromptTemplate(
            input_variables=["question", "answer"],
            template="question: {question}\nanswer: {answer}"
        )
        prompt = FewShotPromptTemplate(
            examples=purchase_prompt,
            example_prompt=example_prompt,
            suffix="question : {question}",
            input_variables=["question"],
        )
        llm = OpenAI2(openai_api_key=api_key)
        parser = StrOutputParser()
        chain = prompt | llm | parser

        chain_answer = chain.invoke({"question": msg})
        try:
            answer_list = ast.literal_eval(str(chain_answer[chain_answer.find('form: ')+6:]))
        except SyntaxError:
            answer_list = ['no_product']

        if len(answer_list) == 1:
            return 'no_product'
        elif len(answer_list) == 3:
            if answer_list[0] == 'no_product' or answer_list[0] == '':
                return 'no_product'
            else:
                return answer_list
        elif answer_list[1] == '' and answer_list[2] == '':
            return 'no_product'
        else:
            return 'no_product'
        
    # 와/과 구분    
    def ends_with_jong(self, kstr):
        k = kstr[-1]
        if "가" <= k <= "힣":
            if (ord(k)-ord("가")) % 28 > 0:
                return "과"
            else:
                return "와"
        else:
            return "와"
        
    # RAG 형식으로 대화 저장
    def answer_to_json(self, type, flag, final_matched_agent, product, price, msg):
        return {
            "type": type,
            "ai_flag": flag,
            "mem_nick_name": final_matched_agent,
            "product_name": product,
            "product_price": price,
            "talk_msg": msg
            }

base_f = base_f(api_key)

In [227]:
class RAG_f:
    def __init__(self):
        self.df = base_f.df
        self.client = base_f.client
        self.Kiwi = Kiwi(typos="basic", model_type='sbg')
        self.stopwords = Stopwords().remove(('사람', 'NNG'))
        self.vectors_list = self.df['Embeddings'].tolist()
        self.index = self.faiss_index(self.vectors_list)

    # 텍스트 벡터값으로 변환 (리스트 형태로)
    def get_embedding(self, text, model="text-embedding-3-large"):
        text = str(text).replace("\n", " ")
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding
    
    # 텍스트 벡터값으로 변환 (array 형식으로 맞추기)
    def to_vector(self, x):
        _vector_ = self.get_embedding(x)
        _vector = np.array([_vector_]).astype(np.float32)
        return _vector

    # 벡터끼리의 정보를 계산해두고 인덱스 지정
    def faiss_index(self, vectors_list):
        l = [len(v) for v in vectors_list]
        vectors = np.array(vectors_list).astype(np.float32)
        self.index = faiss.IndexFlatL2(max(l))
        self.index.add(vectors)
        return self.index
    
    # 메시지의 의도를 파악하는 함수
    def classify(self, msg, matched_agent):
        # 활동명만 입력했을 경우 Match나 Buy&Sell로 연결되는 것을 방지하기 위함
        if msg.replace(" ", "") in matched_agent:
            return ["Chat"]
        else:
            prompt = classify_prompt(msg)
            response = self.client.chat.completions.create(
            model = "gpt-4-1106-preview",
            messages=[{"role": "system", "content": prompt}],
            temperature = 0
            )
            response = response.choices[0].message.content
            try:
                classification = ast.literal_eval(response)
                if classification[0] == "Buy&Sell":
                    return ["Buy&Sell", classification[1]]
                elif classification[0] == "Match":
                    return ["Match", classification[1]]
                else:
                    return ["Chat"]
            except:
                return ["Chat"]

    # 형태소로 분리, 명사 추출
    def tokenize_N(self, text):
        split_s = self.Kiwi.tokenize(text, stopwords=self.stopwords, normalize_coda=True)
        N_list = [i.form for i in split_s if i.tag == "NNG" or i.tag == "NNP"]
        split_list = [i.form for i in split_s]
        split_f = ','.join(split_list).replace(",", " ")
        return split_f, N_list
    
    # keyword가 들어간 문장을 2개 검색
    def keyword_search(self, keyword):
        final_list = []
        k_search = self.df[self.df['메시지'].str.contains(keyword)].index
        if len(k_search) > 1:
            n = 0
            while len(final_list) < 2:
                final_list += [(self.df.iloc[k_search[n], 0], self.df.iloc[k_search[n], 1])]
                n += 1
        elif len(k_search) == 1:
            final_list += [(self.df.iloc[k_search[0], 0], self.df.iloc[k_search[0], 1])]
        else:
            pass
        return final_list

    # 입력 문장과 유사한 문장들 검색
    def extract_sentences(self, indices, exclude_nickname=None):
        extracted_info = []
        for idx in indices.flatten():
            if idx < len(self.df):
                agent_name = self.df.iloc[idx]['Agent']
                message = self.df.iloc[idx]['메시지']
                # 사용자의 nickname을 포함하는 항목을 제외
                if exclude_nickname is None or agent_name != exclude_nickname:
                    extracted_info.append((agent_name, message))
        return extracted_info

    # 기존 list 내에 있는 문장을 제외하고 추가로 검색
    # k1은 처음 검색해올 문장 개수, k2는 최종적으로 출력할 문장 개수(중복된 문장이 몇개인지에 따라 달라짐)
    def search_vector(self, sentence, k1, k2, dp_list):
        list_ = []
        _, indices = self.index.search(self.to_vector(sentence), k=k1)
        extracted_info = self.extract_sentences(indices)
        for info in extracted_info[:k2]:
            if info not in dp_list:
                list_.append(info)
        return list_

    def find_closest_match(self, msg_type, msg_object, user_input, number):
        final_list = []
        try:
            tokenized_input, n_list = self.tokenize_N(user_input)
            n_list = n_list[:number[0]]
            if msg_type == "Buy&Sell":
                for n_word in n_list:
                    final_list += self.keyword_search(n_word)
                final_list = list(set(final_list))
                # final_list.append('---------------keyword------------------')
                final_list += self.search_vector(msg_object, number[1]*2, number[1], final_list)
                # final_list.append('---------------object------------------')
                final_list += self.search_vector(tokenized_input, number[2]*2, number[2], final_list)
                # final_list.append('---------------tokenized----------------')
                _, indices = self.index.search(self.to_vector(user_input), k=number[3])
                extracted_info = self.extract_sentences(indices)
                n = 0
                while len(final_list) < number[3]:
                    if extracted_info[n] not in final_list:
                        final_list.append(extracted_info[n])
                    n += 1
            elif msg_type == "Match":
                final_list += self.search_vector(msg_object, number[1]*2, number[1], final_list)
                # final_list.append('---------------object------------------')
                final_list += self.search_vector(tokenized_input, number[2]*2, number[2], final_list)
                # final_list.append('---------------tokenized----------------')
                _, indices = self.index.search(self.to_vector(user_input), k=number[3])
                extracted_info = self.extract_sentences(indices)
                n = 0
                while len(final_list) < number[3]:
                    if extracted_info[n] not in final_list:
                        final_list.append(extracted_info[n])
                    n += 1
            else:
                for n_word in n_list:
                    final_list += self.keyword_search(n_word)
                final_list = list(set(final_list))
                # final_list.append('---------------keyword------------------')
                final_list += self.search_vector(tokenized_input, number[2]*2, number[2], final_list)
                # final_list.append('---------------tokenized----------------')
                _, indices = self.index.search(self.to_vector(user_input), k=number[3])
                extracted_info = self.extract_sentences(indices)
                n = 0
                while len(final_list) < number[3]:
                    if extracted_info[n] not in final_list:
                        final_list.append(extracted_info[n])
                    n += 1
        except BadRequestError:
            _, indices = self.index.search(self.to_vector(user_input), k=number[3])
            final_list = self.extract_sentences(indices, )
        return final_list

RAG_f = RAG_f()

In [233]:
# RAG 시간체크
RAG_f.find_closest_match('', '', '안녕하세요', [2, 2, 2, 10])

[('마케팅마스터산책가',
  '안녕하세요, 25세 남성입니다. 저는 현재 마케팅로 일하고 있어요. 일상에서는 산책에 몰두하며 삶의 활력을 찾고 있고, 여행을 좋아해 다양한 곳을 방문하는 것도 제 취미 중 하나에요. 저는 스포츠를 좋아하는 사람한 사람과 만나길 희망하고 있어요. 서로를 이해하고 지지해 줄 수 있는 사람이면 좋겠어요. 그리고 자연을 사랑해서 캠핑이나 등산을 자주 가며, 조용한 시간을 가질 때 생각을 정리하는 것도 좋아해요.'),
 ('긍정의 멜로디 마스터',
  '안녕하세요, 29세 남성입니다. 저는 현재 예술로 일하고 있어요. 저의 취미는 음악 듣기인데, 이를 통해 스트레스를 해소하고 새로운 에너지를 얻고 있어요. 더불어, 제 이상형은 긍정적인 사람이고, 서로의 생각을 공유하고 존중하는 관계를 꿈꾸고 있어요. 또한, 일상의 소소한 것들에서 행복을 찾는 것을 좋아하고, 책 읽기나 음악 감상도 자주 해요.'),
 ('식료품요정', '"안녕하세요! 오늘의 신선한 식료품을 안전하게 배달해 드립니다. 특별 요청이 있으신가요?"'),
 ('특별행사달인', '"안녕하세요! 특별한 행사를 위한 준비를 도와드릴게요. 행사에 대한 세부 사항을 알려주시면 감사하겠습니다."'),
 ('세차의달인', '"안녕하세요! 귀하의 차량을 깨끗하고 반짝이게 세차해 드리겠습니다. 선호하는 세차 시간대가 있으신가요?"'),
 ('요리영화남작\n',
  '안녕하세요, 21세 남성입니다. 저는 현재 요리로 일하고 있어요. 제 취미는 영화 감상로, 이것을 통해 새로운 사람들을 만나고 다양한 경험을 하고 있어요. 또한, 제 삶에서 지적인 대화를 즐기는 사람한 파트너를 만나는 것이 꿈입니다. 서로에게 긍정적인 영향을 주고 받을 수 있는 관계가 중요해요. 일상의 소소한 것들에서 행복을 찾는 것을 좋아하고, 책 읽기나 음악 감상도 자주 해요.'),
 ('드라이클리닝마스터', '"안녕하세요! 귀하의 드라이클리닝을 오늘 처리해 드릴게요. 특별한 주의 사항이 있나요?"'),
 ('우편수호자', '

In [228]:
class JarvisJust:
    def __init__(self, classify_method, print_method, number):
        self.client = base_f.client
        self.classify_method = classify_method
        self.print_method = print_method
        self.number = number

    def make_profile(self, msg, matched_agent):
        if self.classify_method == 0:
            profile = RAG_f.find_closest_match('', '', msg, self.number)
            return profile
        elif self.classify_method == 1:
            classification= RAG_f.classify(msg, matched_agent)
            if classification[0] == "Chat":
                return "Chat"
            else:
                profile = RAG_f.find_closest_match(classification[0], classification[1], msg, self.number)
                return profile
    
    def chat(self, msg, state):
        chat_prompt = get_chat_prompt(msg)
        state.append({"role": "system", "content": chat_prompt})
        response = base_f.client.chat.completions.create(
                        model="gpt-4-1106-preview",
                        messages=state,
                        stream=False)
        response = response.choices[0].message.content
        result_ME = base_f.answer_to_json('chat', 'M', '', '', 0, msg)
        result_AI = base_f.answer_to_json('chat', 'A', '', '', 0, response)
        state.append({"role": "assistant", "content": response})
        return response, result_ME, result_AI, state

    def print_0(self, profile, msg, state):
        def find_agent(msg, profile):
            RAG_prompt = matching_rag_prompt(msg, profile)
            response = base_f.client.chat.completions.create(
                            model = "gpt-4-1106-preview",
                            messages=[{"role": "system", "content": RAG_prompt}],
                            temperature = 0
                        )
            response = response.choices[0].message.content
            return response
        def agent_print_form(agent, profile):
            for i in profile:
                if i[0] == agent:
                    msg = i[1]
                    text = f"""사용자의 요구사항을 바탕으로 다음 에이전트를 추천합니다.\n\nAgent: {agent}\n메시지: {msg}"""
                    return text
        def Matching_print(profile, msg):
            print_agent = find_agent(msg, profile)
            if print_agent == 'None':
                final_answer = base_f.unmatched_msg
            else:
                final_answer = agent_print_form(print_agent, profile)
            return final_answer
        def printing(profile, msg, state):
            response =  Matching_print(profile, msg)
            if response == base_f.unmatched_msg:
                result_ME = base_f.answer_to_json('chat', 'M', '', '', 0, msg)
                result_AI = base_f.answer_to_json('chat', 'A', '', '', 0, response)
            else:
                result_ME = base_f.answer_to_json('match', 'M', '', '', 0, msg)
                result_AI = base_f.answer_to_json('match', 'A', '', '', 0, response)
            state.append({"role": "system", "content": f"매칭리스트: {profile}"})
            state.append({"role": "assistant", "content": response})
            return response, result_ME, result_AI, state        
        return printing(profile, msg, state)
    
    def print_1(self, profile, msg, state):
        state.append({"role": "system", "content": RAG_prompt + f" 사용자: <{msg}>. 매칭리스트: {profile}"})
        response = base_f.client.chat.completions.create(
                        model="gpt-4-1106-preview",
                        messages=state,
                        stream=False)
        response = response.choices[0].message.content
        del state[-1]
        state.append({"role": "system", "content": f"매칭리스트: {profile}"})
        state.append({"role": "assistant", "content": response})
        if response == base_f.unmatched_msg:
            result_ME = base_f.answer_to_json('chat', 'M', '', '', 0, msg)
            result_AI = base_f.answer_to_json('chat', 'A', '', '', 0, response)
        else:
            result_ME = base_f.answer_to_json('chat', 'M', '', '', 0, msg)
            result_AI = base_f.answer_to_json('match', 'A', '', '', 0, response)
        return response, result_ME, result_AI, state
    
    def final_print(self, profile, msg, state):
        if self.print_method == 0:
            return self.print_0(profile, msg, state)
        elif self.print_method == 1:
            return self.print_1(profile, msg, state)

    def message_process(self, msg, state, matched_agent, matched_msg):
        state.append({"role": "user", "content": msg})
        if not msg:
            empty_msg = '대화를 입력해주세요.'
            state.append({"role": "assistant", "content": empty_msg})
            result_ME = base_f.answer_to_json('chat', 'M', '', '', 0, msg)
            result_AI = base_f.answer_to_json('chat', 'A', '', '', 0, empty_msg)
            return "end", empty_msg, result_ME, result_AI, state
        else:
            for i in range(len(matched_agent)):
                if matched_agent[i] in msg.replace(" ", ""):
                    classified_process = base_f.classify_process(msg)
                    # 채팅창 연결 process로 이동, 최종 연결된 Agent 및 메시지 return
                    if classified_process == 'Match':
                        consonant = base_f.ends_with_jong(matched_agent[i])
                        connection_msg = f"네 {matched_agent[i]+consonant} 연결해드리겠습니다."
                        state.append({"role": "assistant", "content": connection_msg})
                        result_ME = base_f.answer_to_json('connect', 'M', matched_agent[i], '', 0, msg)
                        result_AI = base_f.answer_to_json('connect', 'A', matched_agent[i], '', 0, connection_msg)
                        return "end", connection_msg, result_ME, result_AI, state
                    # 결제 process로 이동, 최종 연결된 Agent 및 메시지 return
                    elif classified_process == 'Buy&Sell':
                        message = matched_msg[i]
                        product_keyword = base_f.purchase_keyword(api_key, message)
                        if product_keyword == 'no_product':
                            answer_msg = '결제할 상품이 없습니다.'
                            state.append({"role": "assistant", "content": answer_msg})
                            result_ME = base_f.answer_to_json('chat', 'M', '', '', 0, msg)
                            result_AI = base_f.answer_to_json('chat', 'A', '', '', 0, answer_msg)
                            return "end", answer_msg, result_ME, result_AI, state
                        else:
                            pay_nickname = matched_agent[i]
                            pay_product_name =  product_keyword[0]
                            pay_product_price = product_keyword[1]
                            pay_product_quantity = product_keyword[2]
                            pay_msg = "결제를 진행합니다"
                            state.append({"role": "assistant", "content": pay_msg})
                            result_ME = base_f.answer_to_json('sale', 'M', pay_nickname, pay_product_name, pay_product_price, msg)
                            result_AI = base_f.answer_to_json('sale', 'A', pay_nickname, pay_product_name, pay_product_price, pay_msg)
                        return "end", pay_msg, result_ME, result_AI, state

        if "연결" in msg or "매칭" in msg:
            consonant = base_f.ends_with_jong(matched_agent[-1])
            connection_msg = f"네 {matched_agent[-1]+consonant} 연결해드리겠습니다."
            state.append({"role": "assistant", "content": connection_msg})
            result_ME = base_f.answer_to_json('connect', 'M', matched_agent[-1], '', 0, msg)
            result_AI = base_f.answer_to_json('connect', 'A', matched_agent[-1], '', 0, connection_msg)
            return "end", connection_msg, result_ME, result_AI, state
        elif "결제" in msg or "구매할래" in msg:
            message = matched_msg[-1]
            product_keyword = base_f.purchase_keyword(api_key, message)
            if product_keyword == 'no_product':
                answer_msg = '결제할 상품이 없습니다.'
                state.append({"role": "assistant", "content": answer_msg})
                result_ME = base_f.answer_to_json('chat', 'M', '', '', 0, msg)
                result_AI = base_f.answer_to_json('chat', 'A', '', '', 0, answer_msg)
                return "end", answer_msg, result_ME, result_AI, state
            else:
                pay_nickname = matched_agent[-1]
                pay_product_name =  product_keyword[0]
                pay_product_price = product_keyword[1]
                pay_product_quantity = product_keyword[2]
                pay_msg = "결제를 진행합니다"
                state.append({"role": "assistant", "content": pay_msg})
                result_ME = base_f.answer_to_json('sale', 'M', pay_nickname, pay_product_name, pay_product_price, msg)
                result_AI = base_f.answer_to_json('sale', 'A', pay_nickname, pay_product_name, pay_product_price, pay_msg)
            return "end", pay_msg, result_ME, result_AI, state
        # 통상적인 경우 "next" 문자열을 통해 다음 process 실행
        else:
            return "next", '_', '_', '_', state
        
    def final_process(self, state, matched_agent, matched_msg, user_name):

        sub_matched_agent = re.findall(r'Agent: (.*?)\n', state[-1]['content'])
        for i in range(len(sub_matched_agent)):
            # if sub_matched_agent[i] not in matched_agent:
            matched_agent.append(sub_matched_agent[i])
        if self.print_method == 0:
            sub_matched_msg = re.findall(r'메시지: (.*?)$', state[-1]['content'])
            for i in range(len(sub_matched_msg)):
                matched_msg.append(sub_matched_msg[i])
        elif self.print_method == 1:
            sub_matched_msg = re.findall(r'메시지: (.*?)\n', state[-1]['content'])
            for i in range(len(sub_matched_msg)):
                matched_msg.append(sub_matched_msg[i])

        matched_agent = matched_agent[-14:]
        matched_msg = matched_msg[-14:]
        del state[0]
        state = state[-9:]
        state.insert(0, generate_state_message(user_name))
        return state, matched_agent, matched_msg

In [229]:
JarvisJust = JarvisJust(1, 1, [2, 2, 2, 10])

In [230]:
def go_JJ(user_name):
    user_name = user_name
    state = [generate_state_message(user_name)]
    matched_agent = []
    matched_msg = []
    history = []
    print(generate_welcome_message(user_name))
    # if __name__=='__main__':
    while True:
        msg = input("사용자: ")
        if msg == 'break':
            break
        route, answer, result_ME, result_AI, state = JarvisJust.message_process(msg, state, matched_agent, matched_msg)
        if route == "next":
            profile = JarvisJust.make_profile(msg, matched_agent)
            if profile == "Chat":
                answer, result_ME, result_AI, state = JarvisJust.chat(msg, state)
            else:
                answer, result_ME, result_AI, state = JarvisJust.final_print(profile, msg, state)
        state, matched_agent, matched_msg = JarvisJust.final_process(state, matched_agent, matched_msg, user_name)
        print(answer)
        history.append(result_ME)
        history.append(result_AI)
    return history, state

In [231]:
go_JJ("윤이지")

안녕하세요? 윤이지님. 
                        저는 JarvisJust(JJ)입니다. 무엇을 도와드릴까요? 

                        찾으시는 내용을 아래처럼 입력해주시면 JJ가 연결해드립니다.
                        예1) 진미채 살 수 있는 곳을 찾아줘
                        예2) 주말에 축구 같이할 사람 찾아줘

                        연결된 이후 상품 구매를 희망하시면,                                        
                        ‘닉네임과 상품 결제해줘’를
                        예) OOO(닉네임) 상품 결제해줘

                        1:1채팅 연결을 원하시면, 
                        ‘닉네임과 연결해줘’를 입력해주세요.
                        예) OOO(닉네임) 연결해줘
사용자의 요구사항을 바탕으로 다음 에이전트를 추천합니다.

Agent: 파스타마에스트로
메시지: 이탈리아 요리를 좋아하는 분들, 함께 파스타와 피자 만들기에 도전해요! 주 1회 만나서 다양한 이탈리아 요리를 만들고 싶어요.
매칭이유: 파스타를 같이 만들고 싶어하는 분을 찾고 있습니다.
결제할 상품이 없습니다.
네 파스타마에스트로와 연결해드리겠습니다.
사용자의 요구사항을 바탕으로 다음 에이전트를 추천합니다.

Agent: 호두마켓왕
메시지: 맛있고 영양가 높은 호두를 지금 바로 구매하세요! 1kg과 3kg 팩 옵션으로 준비되어 있으니, 건강한 스넥을 찾고 계시다면 이 기회를 놓치지 마세요. 방문하셔서 최고의 호두를 만나보실 수 있습니다. 위치는 중부시장(서울특별시 중구 을지로30길 29)에서 여러분을 기다리고 있습니다.
매칭이유: 호두 및 건강한 스넥을 제공하며, 고객이 방문하여 직접 구매할 수 있는 곳을 찾고 있습니다.
결제를 진행합니다


([{'type': 'chat',
   'ai_flag': 'M',
   'mem_nick_name': '',
   'product_name': '',
   'product_price': 0,
   'talk_msg': '파스타 같이 먹을 사람 찾아줘'},
  {'type': 'match',
   'ai_flag': 'A',
   'mem_nick_name': '',
   'product_name': '',
   'product_price': 0,
   'talk_msg': '사용자의 요구사항을 바탕으로 다음 에이전트를 추천합니다.\n\nAgent: 파스타마에스트로\n메시지: 이탈리아 요리를 좋아하는 분들, 함께 파스타와 피자 만들기에 도전해요! 주 1회 만나서 다양한 이탈리아 요리를 만들고 싶어요.\n매칭이유: 파스타를 같이 만들고 싶어하는 분을 찾고 있습니다.'},
  {'type': 'chat',
   'ai_flag': 'M',
   'mem_nick_name': '',
   'product_name': '',
   'product_price': 0,
   'talk_msg': '파스타마에스트로거 결제해줘'},
  {'type': 'chat',
   'ai_flag': 'A',
   'mem_nick_name': '',
   'product_name': '',
   'product_price': 0,
   'talk_msg': '결제할 상품이 없습니다.'},
  {'type': 'connect',
   'ai_flag': 'M',
   'mem_nick_name': '파스타마에스트로',
   'product_name': '',
   'product_price': 0,
   'talk_msg': '연결해줘'},
  {'type': 'connect',
   'ai_flag': 'A',
   'mem_nick_name': '파스타마에스트로',
   'product_name': '',
   'product_price': 0,
   'talk_msg': '네 파